Quitar el fondo del video.

In [ ]:
import cv2
import numpy as np
import os

def quitar_fondo_negro(directorio_entrada, directorio_salida):
    # Crear el directorio de salida si no existe
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)
    
    # Procesar cada archivo en el directorio de entrada
    for nombre_archivo in os.listdir(directorio_entrada):
        ruta_entrada = os.path.join(directorio_entrada, nombre_archivo)
        
        if os.path.isfile(ruta_entrada) and nombre_archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Leer la imagen
            imagen = cv2.imread(ruta_entrada)
            # Convertir a formato RGBA (con canal alfa para transparencia)
            imagen_rgba = cv2.cvtColor(imagen, cv2.COLOR_BGR2BGRA)
            
            # Crear una máscara para los píxeles negros
            lower_black = np.array([0, 0, 0, 0], dtype=np.uint8)
            upper_black = np.array([50, 50, 50, 255], dtype=np.uint8)
            mask = cv2.inRange(imagen_rgba, lower_black, upper_black)
            
            # Establecer los píxeles negros en transparentes
            imagen_rgba[mask == 255] = [0, 0, 0, 0]
            
            # Guardar la imagen sin fondo en el directorio de salida
            ruta_salida = os.path.join(directorio_salida, nombre_archivo)
            cv2.imwrite(ruta_salida, imagen_rgba)
            print(f"Procesado: {nombre_archivo}")
# Especifica el directorio de entrada y de salida
directorio_entrada = './P5 - Imagenes/happy/'
directorio_salida = './P5 - Imagenes/happy_out/'

# Ejecuta la función
quitar_fondo_negro(directorio_entrada, directorio_salida)

Primera aproximación

In [11]:
import cv2
from deepface import DeepFace
import numpy as np
from mtcnn.mtcnn import MTCNN
import dlib
import glob
import random

In [ ]:
# Cargar la imagen del halo para la emoción 'happy'
happy_filter = cv2.imread("./P5 - Imagenes/corona.png", cv2.IMREAD_UNCHANGED)
print("Happy filter loaded:", happy_filter is not None)
if happy_filter is not None:
    print("Forma de happy_filter:", happy_filter.shape)

# Cargar los frames de la animación para superponer en toda la ventana
happy_animation_frames = []
for filename in sorted(glob.glob('./P5 - Imagenes/happy/*.png')):
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if img is not None:
        happy_animation_frames.append(img)
print(f"Loaded {len(happy_animation_frames)} frames for happy animation over the full window.")

# Inicializar el detector de rostros y el predictor de forma de dlib
detector_dlib = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Inicializar la captura de video
cap = cv2.VideoCapture(0)

def overlay_image_alpha(img, img_overlay, x, y):
    """Superpone img_overlay sobre img en las coordenadas (x, y) utilizando el canal alpha de img_overlay."""
    h, w = img_overlay.shape[:2]

    if x >= img.shape[1] or y >= img.shape[0]:
        return

    # Recortar si la imagen overlay se sale de los límites
    if x + w > img.shape[1]:
        w = img.shape[1] - x
        img_overlay = img_overlay[:, :w]

    if y + h > img.shape[0]:
        h = img.shape[0] - y
        img_overlay = img_overlay[:h]

    # Comprobar si la imagen overlay tiene canal alpha
    if img_overlay.shape[2] == 4:
        # Separar los canales de color y alpha
        overlay_img = img_overlay[:, :, :3]
        overlay_mask = img_overlay[:, :, 3:] / 255.0
    else:
        overlay_img = img_overlay
        overlay_mask = np.ones((h, w, 1), dtype=float)

    # Extraer la región de interés del frame original
    img_region = img[y:y+h, x:x+w]

    # Combinar la imagen overlay con la región de interés
    img[y:y+h, x:x+w] = (1.0 - overlay_mask) * img_region + overlay_mask * overlay_img

def is_smiling(landmarks_points):
    # Puntos de la boca
    mouth_left = np.array(landmarks_points[48])  # Esquina izquierda de la boca
    mouth_right = np.array(landmarks_points[54])  # Esquina derecha de la boca
    mouth_top = np.array(landmarks_points[51])  # Punto superior de la boca
    mouth_bottom = np.array(landmarks_points[57])  # Punto inferior de la boca

    # Distancias
    mouth_width = np.linalg.norm(mouth_right - mouth_left)
    mouth_height = np.linalg.norm(mouth_bottom - mouth_top)

    # Relación entre ancho y alto de la boca
    ratio = mouth_width / mouth_height
    print(f"Smile ratio: {ratio}")

    # Umbral para determinar si hay una sonrisa
    return ratio < 2.7  # Ajusta este valor según sea necesario

def crear_haz_cono_suave(ancho, alto, origen_x, direccion, color=(0, 255, 255), opacidad=0.3):
    """Crea una imagen con un haz de luz en forma de cono con bordes suaves, con un ancho un 20% mayor."""
    # Crear una imagen con fondo transparente
    haz = np.zeros((alto, ancho, 4), dtype=np.uint8)

    # Definir los puntos del cono
    vertice_superior = (origen_x, 0)
    
    # Aumentar el ancho de la base del cono un 20%
    ancho_extra = 100
    
    if direccion == "derecha":
        vertice_inferior_izq = (origen_x - int(ancho * 0.15) + ancho_extra, alto)
        vertice_inferior_der = (origen_x + int(ancho * 0.35) + ancho_extra, alto)
    else:  # direccion == "izquierda"
        vertice_inferior_izq = (origen_x - int(ancho * 0.35) - ancho_extra, alto)
        vertice_inferior_der = (origen_x + int(ancho * 0.15) - ancho_extra, alto)
    
    # Crear la máscara del cono
    mask = np.zeros((alto, ancho), dtype=np.uint8)
    vertices = np.array([vertice_superior, vertice_inferior_izq, vertice_inferior_der], np.int32)
    cv2.fillPoly(mask, [vertices], 255)
    
    # Crear un gradiente vertical
    gradient = np.linspace(1.0, 0.0, alto)
    gradient = np.repeat(gradient[:, np.newaxis], ancho, axis=1)
    
    # Aplicar el gradiente a la máscara
    alpha = (mask / 255.0) * gradient * opacidad
    
    # Crear la imagen del haz con el color especificado
    color_array = np.full((alto, ancho, 3), color, dtype=np.uint8)
    
    # Combinar el color y el alpha
    haz[:, :, :3] = color_array
    haz[:, :, 3] = (alpha * 255).astype(np.uint8)
    
    return haz

def overlay_conos(frame, haz_list):
    """Alterna los haces sobre el frame para crear el efecto de discoteca con dos haces seleccionados aleatoriamente."""
    # Número total de haces
    num_haces = len(haz_list)

    # Selecciona dos índices aleatorios diferentes
    indice_haz1 = random.randint(0, num_haces - 1)
    indice_haz2 = random.randint(0, num_haces - 1)
    
    # Asegurarse de que no sean el mismo índice
    while indice_haz2 == indice_haz1:
        indice_haz2 = random.randint(0, num_haces - 1)

    # Superpone ambos haces en el frame
    overlay_image_alpha(frame, haz_list[indice_haz1], 0, 0)
    overlay_image_alpha(frame, haz_list[indice_haz2], 0, 0)

# Obtener el tamaño del frame de la cámara
ret, frame = cap.read()
if not ret:
    print("Error al capturar el frame de la cámara")
    exit()
alto_frame, ancho_frame = frame.shape[:2]

haces = []

for i in range(10):
    # Distribuir los conos a lo largo del ancho del frame
    posicion_x = int((i + 1) * ancho_frame / 11)
    # Alternar la dirección entre derecha e izquierda
    direccion = "derecha" if i % 2 == 0 else "izquierda"
    haz = crear_haz_cono_suave(ancho_frame, alto_frame, origen_x=posicion_x, direccion=direccion)
    haces.append(haz)


frame_counter = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir el frame a escala de grises para dlib
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros con dlib
    faces = detector_dlib(gray)

    detected_emotion = 'neutral'

    for face in faces:
        landmarks = predictor(gray, face)
        landmarks_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(68)]

        # Dibujar los puntos faciales en el frame (opcional)
        #for point in landmarks_points:
        #    cv2.circle(frame, point, 2, (0, 255, 0), -1)

        # Detectar si la persona está sonriendo
        if is_smiling(landmarks_points):
            detected_emotion = 'happy'
            print("Emoción detectada:", detected_emotion)

            # Aplicar el filtro 'happy' sobre la cabeza
            if happy_filter is not None:
                # Calcular la posición para el filtro sobre la cabeza
                eyebrow_points = landmarks_points[17:27]
                min_eyebrow_y = min([point[1] for point in eyebrow_points])
                leftmost_point = landmarks_points[0]
                rightmost_point = landmarks_points[16]
                head_width = int(np.linalg.norm(np.array(leftmost_point) - np.array(rightmost_point)))

                halo_width = head_width
                halo_height = int(happy_filter.shape[0] * (halo_width / happy_filter.shape[1]))
                resized_halo = cv2.resize(happy_filter, (halo_width, halo_height), interpolation=cv2.INTER_AREA)

                face_center_x = (leftmost_point[0] + rightmost_point[0]) // 2
                x_offset = face_center_x - halo_width // 2
                y_offset = min_eyebrow_y - halo_height - 20  # Ajustar este valor según sea necesario

                x_offset = max(x_offset, 0)
                y_offset = max(y_offset, 0)

                overlay_image_alpha(frame, resized_halo, x_offset, y_offset)

            # Superponer los frames de la animación sobre toda la ventana
            if len(happy_animation_frames) > 0:
                # Obtener el frame actual de la animación
                animation_frame = happy_animation_frames[frame_counter % len(happy_animation_frames)]
                frame_counter += 1

                # Redimensionar el frame de la animación para que coincida con el tamaño de la ventana
                resized_animation_frame = cv2.resize(animation_frame, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_AREA)

                # Superponer la animación sobre el frame
                overlay_image_alpha(frame, resized_animation_frame, 0, 0)
            
            # Agregar efecto de luces de discoteca
            overlay_conos(frame, haces)
            frame_counter += 1
        else:
            detected_emotion = 'neutral'
            print("Emoción detectada:", detected_emotion)

    

    # Mostrar el frame con los efectos
    cv2.imshow('Filtro de Felicidad', frame)

    # Presionar 'q' para salir
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar ventanas
cap.release()
cv2.destroyAllWindows()

Happy filter loaded: True
Forma de happy_filter: (500, 500, 4)
Loaded 520 frames for happy animation over the full window.
Smile ratio: 4.057059470062543
Emoción detectada: neutral
Smile ratio: 3.8859582011312983
Emoción detectada: neutral
Smile ratio: 4.076723210023896
Emoción detectada: neutral
Smile ratio: 3.6677387710015137
Emoción detectada: neutral
Smile ratio: 3.378515153271327
Emoción detectada: neutral
Smile ratio: 2.9413267838903456
Emoción detectada: neutral
Smile ratio: 4.605165270629419
Emoción detectada: neutral
Smile ratio: 2.716072381275556
Emoción detectada: neutral
Smile ratio: 2.4470642646729632
Emoción detectada: happy
Smile ratio: 1.9000641035900356
Emoción detectada: happy
Smile ratio: 1.8339516582525441
Emoción detectada: happy
Smile ratio: 2.2933144726633596
Emoción detectada: happy
Smile ratio: 1.9738766994684183
Emoción detectada: happy
Smile ratio: 1.973859641326229
Emoción detectada: happy
Smile ratio: 2.1144787556661013
Emoción detectada: happy
Smile ratio: